In [730]:
import numpy as np
import pandas as pd
import csv
from io import StringIO # import StringIO module
import glob
import os

In [731]:
def get_df_5_partidos(link):

    with open(link, "r") as f:
        text = f.read() # read the file as a string

    # replace the substring in the whole string
    text = text.replace('COMPROMISO,', 'COMPROMISO')
    text = text.replace('COMPROMISO.', 'COMPROMISO')

    # convert the string to a list of lines
    rows = text.splitlines()

    start = [i for i, s in enumerate(rows) if "Código ,Nombre" in s][0]

    data = rows[start:]

    # join the data lines into a single string
    data_str = '\n'.join(data)

    # convert the data string to a StringIO object
    data_io = StringIO(data_str)

    # read the data into a dataframe with comma separator
    df = pd.read_csv(data_io, sep=',',dtype=str,  names=['col1', 'col2', 'col3', 'col4', 'col5', 'col6','col7', 'col8', 'col9','col10'], quoting=csv.QUOTE_NONE)
    df = df.iloc[1:25].copy()
    lenght = len(df.columns)
    result = df.iloc[:,2].fillna('')

    for i in range(3,lenght):
        result = result + df.iloc[:,i].fillna('')
    eliminate_special_characters = {' ':'', '\.':'','/':'1','o':'0','O':'0', ':':'','C':'0','-':'', 'X':'','a':'9',
                                    '\(':'','\)':'','I':'1', 'U':'0', 'G':'6','D':'0','S':'5','B':'8','E':'8','A':'4',
                                    'b':'6','s':'5','\#':'4', '\+':'7', 'Q':'0', "\\\\":'1', 'H':'4', '&':'8', '\>':'',
                                    '\$':'5', 'x':'','\?':'3', 'J':'','T':'7', 'n':'6', 't':'7','\<':'5', '\}':'3',
                                    'Z':'2'}
    df["votos_ocr"] = result.replace(eliminate_special_characters, regex=True)
    df = df[["col1","votos_ocr"]].copy()
    df.columns = ["codigo","votos_ocr"]
    #df_final = df[df["codigo"].str.contains("23") |df["codigo"].str.contains("15")|df["codigo"].str.contains("41")|df["codigo"].str.contains("88")|df["codigo"].str.contains("20") ].copy()

    df_final = df[df["codigo"].str.contains("41") |df["codigo"].str.contains("20") ].copy()

    df_final["codigo"] = df_final["codigo"].str.rstrip()
    change_numebers = {"23":"TODOS","15":"UNE","41":"VALOR UNIONISTA","88":"FOPPA X LA CIUDAD","20":"CREO"}
    df_final["codigo"] = df_final["codigo"].replace(change_numebers)

    return df_final

In [732]:
directorio_de_archivos = glob.glob("/Users/rudikrompich/Git/recopilacion_y_analisis_elecciones_de_guatemala_2023/final_data/elecciones_2023/corporacion_municipal/distrito_central_csv/*.*")

In [733]:
len(directorio_de_archivos)

2023

In [734]:
cantidad_errores = 0
df_general_ocr = pd.DataFrame()
for archivo in directorio_de_archivos:
    filename = os.path.basename(archivo).split(".")[0]
    name_separated = filename.split("_")
    centro = name_separated[2]
    mesa = name_separated[4]

    try:
        df_individual = get_df_5_partidos(archivo)
        df_individual["centro"] = centro
        df_individual["mesa"] = mesa
        df_individual["centro"] = df_individual["centro"].astype(int)
        df_individual["mesa"] = df_individual["mesa"].astype(int)
        df_general_ocr = pd.concat([df_general_ocr,df_individual])
    except:
        print(archivo)
        cantidad_errores = cantidad_errores + 1

In [735]:
df_general_ocr

,codigo,votos_ocr,centro,mesa
3,VALOR UNIONISTA,64,53,693
24,CREO,039,53,693
3,VALOR UNIONISTA,815,72,933
24,CREO,058,72,933
3,VALOR UNIONISTA,41,111,1512
...,...,...,...,...
24,CREO,051,81,1034
3,VALOR UNIONISTA,076,70,892
24,CREO,064,70,892
3,VALOR UNIONISTA,18,29,366


In [736]:
dataframe_generado = sorted(df_general_ocr["mesa"].unique().tolist())

all_numbers = set(range(1, 2024))

missing_numbers = all_numbers - set(dataframe_generado)
missing_numbers

set()

In [737]:
sorted(df_general_ocr["mesa"].unique().tolist())

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

In [738]:
analisis = df_general_ocr.groupby(["mesa"]).count()
analisis[analisis["codigo"] != 2]

,codigo,votos_ocr,centro
mesa,,,


# Datos del TSE del sistema TREP

In [739]:
df_complete_trep = pd.read_csv("/Users/rudikrompich/Git/recopilacion_y_analisis_elecciones_de_guatemala_2023/data_sistema_trep/4-CORPORACIÓN/gtm2023_e4d00m01.csv", sep=",",skiprows=4)
df_complete_trep = df_complete_trep.iloc[:,:31].copy().drop(columns=["DEPARTAMENTO", "ID_DEPARTAMENTO", "MUNICIPIO", "ID_MUNICIPIO", "PADRÓN"])
df_complete_trep = pd.melt(df_complete_trep, id_vars=["CENTRO_DE_VOTACIÓN", "MESA"], value_vars=df_complete_trep.columns[2:], var_name="codigo", value_name="votos_trep")

df_complete_trep["votos_trep"] = df_complete_trep["votos_trep"].replace("--", 0)
df_complete_trep["votos_trep"] = df_complete_trep["votos_trep"].astype(int)
df_complete_trep.columns = ['centro', 'mesa', 'codigo', 'votos_trep']
df_complete_trep

,centro,mesa,codigo,votos_trep
0,1,1,TODOS,9
1,1,2,TODOS,14
2,1,3,TODOS,10
3,1,4,TODOS,9
4,2,5,TODOS,7
...,...,...,...,...
48547,152,2019,CREO,2
48548,152,2020,CREO,1
48549,152,2021,CREO,2
48550,152,2022,CREO,4


# Merge

In [740]:
def check_votos(row):
  # convert votos_ocr column to int if possible, otherwise return "Incorrect"
  try:
    row["votos_ocr"] = int(row["votos_ocr"])
  except ValueError:
    return "Incorrect"

  # compare votos_ocr and votos_trep columns and return "correct" if they match, otherwise return "Incorrect"
  if row["votos_ocr"] == row["votos_trep"]:
    return "correct"
  else:
    return "Incorrect"

def change_votos(row):
  # convert votos_ocr column to int if possible, otherwise return "Incorrect"
  try:
    row["votos_ocr"] = int(row["votos_ocr"])
  except ValueError:
    return np.NaN

  # compare votos_ocr and votos_trep columns and return "correct" if they match, otherwise return "Incorrect"
  if row["votos_ocr"] == row["votos_trep"]:
    return row["votos_trep"]
  else:
    return np.NaN


In [741]:
df_completo_con_ocr_y_trep = pd.merge(df_general_ocr, df_complete_trep, on=["centro", "mesa", "codigo"],how='left')
df_completo_con_ocr_y_trep['votos_trep'] = df_completo_con_ocr_y_trep['votos_trep']

df_completo_con_ocr_y_trep["result"] = df_completo_con_ocr_y_trep.apply(check_votos, axis=1)
df_completo_con_ocr_y_trep["votos_ocr_correcto"] = df_completo_con_ocr_y_trep.apply(change_votos, axis=1)


df_completo_con_ocr_y_trep.loc[df_completo_con_ocr_y_trep["result"] == "Incorrect", "votos_ocr"] = df_completo_con_ocr_y_trep.loc[df_completo_con_ocr_y_trep["result"] == "Incorrect", "votos_ocr"].replace({'1':''},regex=True)

df_completo_con_ocr_y_trep["result"] = df_completo_con_ocr_y_trep.apply(check_votos, axis=1)
df_completo_con_ocr_y_trep["votos_ocr_correcto"] = df_completo_con_ocr_y_trep.apply(change_votos, axis=1)

df_completo_con_ocr_y_trep.sort_values(by="mesa").reset_index(drop=True,inplace=True)
df_completo_con_ocr_y_trep = df_completo_con_ocr_y_trep[['centro', 'mesa','codigo', 'result', 'votos_trep','votos_ocr','votos_ocr_correcto']].copy()

In [742]:
df_completo_con_ocr_y_trep["result"].value_counts()

correct      3550
Incorrect     496
Name: result, dtype: int64

In [743]:
df_completo_con_ocr_y_trep.to_excel('/Users/rudikrompich/Git/recopilacion_y_analisis_elecciones_de_guatemala_2023/data_analisis/errores_corporacion_municipal_distrito_central.xlsx',index=False)

# Análisis

In [744]:
df_analisis_con_correcciones = pd.read_excel("/Users/rudikrompich/Git/recopilacion_y_analisis_elecciones_de_guatemala_2023/data_analisis/arreglo_manual.xlsx")
df_analisis_con_correcciones

,centro,mesa,codigo,result,votos_trep,votos_ocr,votos_ocr_correcto,comentario
0,2,9,VALOR UNIONISTA,Incorrect,52,5,52,NaN
1,2,10,CREO,Incorrect,47,04,47,NaN
2,2,11,CREO,Incorrect,57,NaN,57,NaN
3,2,11,VALOR UNIONISTA,Incorrect,65,NaN,65,NaN
4,2,16,CREO,Incorrect,59,0549,59,NaN
...,...,...,...,...,...,...,...,...
4041,85,1137,VALOR UNIONISTA,correct,127,127,127,NaN
4042,86,1138,VALOR UNIONISTA,correct,127,127,127,NaN
4043,152,2023,CREO,Incorrect,0,NaN,0,NaN
4044,83,1091,VALOR UNIONISTA,correct,132,132,132,NaN


In [745]:
df_analisis_con_correcciones["diferencia"] = df_analisis_con_correcciones["votos_trep"] - df_analisis_con_correcciones["votos_ocr_correcto"]


In [746]:
df_final_diferencias = df_analisis_con_correcciones[(df_analisis_con_correcciones["diferencia"] != 0) |(df_analisis_con_correcciones["comentario"].notna())].copy()
df_final_diferencias

,centro,mesa,codigo,result,votos_trep,votos_ocr,votos_ocr_correcto,comentario,diferencia
8,47,604,CREO,correct,0,0,59,Se confundieron de papeleta lol https://www.tr...,-59
22,102,1335,CREO,correct,1,01,91,Este se arregló por medio del algoritmo en est...,-90
32,152,2015,CREO,Incorrect,2,00,2,Gran desorden!,0
36,11,153,VALOR UNIONISTA,Incorrect,10,NaN,10,No aparecen los datos para los primeros 6 part...,0
41,34,430,VALOR UNIONISTA,Incorrect,77,NaN,77,Se digitó mal,0
159,19,242,VALOR UNIONISTA,Incorrect,0,049,49,NaN,-49
161,19,242,CREO,Incorrect,0,046,45,NaN,-45
255,30,368,VALOR UNIONISTA,Incorrect,0,P,69,Se confundieron de papeleta lol,-69
274,30,368,CREO,Incorrect,0,024,50,Se confundieron de papeleta lol,-50
602,47,599,VALOR UNIONISTA,Incorrect,0,045,45,Esta papeleta súper rara thoug,-45


In [747]:
df_analisis_con_correcciones.groupby(["codigo"]).sum()

/var/folders/5x/zy_k3bv51t34gj2y4llrp6lw0000gn/T/ipykernel_23278/1411082501.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_analisis_con_correcciones.groupby(["codigo"]).sum()


,centro,mesa,votos_trep,votos_ocr_correcto,diferencia
codigo,,,,,
CREO,152812,2047276,105698,106335,-637
VALOR UNIONISTA,152812,2047276,106220,106579,-359


In [749]:
106579-106335

244